# Capstone Check-In Two

## Proposal
My capstone project is the creation of a protein-binding predictive model using primary structure.  Originally, I hoped to create a regressor that predicts the protein binding affinity for an inputted pair of proteins.  But, I have simplified the scope to a binary classification: interacts or doesn't interact.  Such a predictive model could be used to determine if newly sequenced viral proteins, like those on the surface of novel coronavirus, bind to human membrane receptors.  Identifying protein-protein interactions without laboratory processes is an important application of machine learning.

## Problem Statement
SARS-COV-2 has infected at least 30 million people worldwide.  The global efforts to find a vaccine, treat COVID-19, and understand the virus can only be described as the largest scientific project every undertaken.  One branch of this project is mapping interactions between SARS-COV-2 surface proteins to human membrane receptors.  The goal of this project is to train a binary classifier to identify whether two inputted proteins interact or not.  The model will be applied to recently sequenced SARS-COV-2 surface proteins to determine their likely human targets.

## Methods and Models
The model will be trained on the primary structures (amino acid sequences) of thousands of protein profiles scraped from the UniProt protein database.  We are not interested in inferences (e.g. "presence of cysteines in both molecules increases the chance of interaction"), we are interested in accuracy.  Considering the complexity of protein data, we should move immediately to deep learning to extract as much predicting strength from the amino acid strings as possible.  I anticipate tuning a Convolutional Neural Network

## Risks and Assumptions
Predicting chemical forces between macromolecules is a vastly complex task that often involves computationally exhausting 3D modeling.  Limiting our model training to a 1D amino acid string avoids this problem, but also lowers our expectations for the model's performance.  However, we are still expecting the model to be able to intuit enough chemical understanding from just amino acids to predict binding.  This is a huge assumption the relies on the strength of deep learning and the size of our training data.  A potential risk of the project is that the model entirely fails to discern meaningful information from primary structure and secondary structure data (presence of alpha helices, beta sheets, etc.) must be added to the training set.

## Data Source
The commented code below includes saved functions that query the RCSB Protein DataBase. I originally scraped from this site, but ended up switching to UniProt. I've kept the code below, just in case I want to gather corroborating data later on.

I customized advanced queries from UniProt to only select human proteins with at least one interactor.  I choose to include protein mass, length, sequence, and interactors as my columns.

https://www.uniprot.org/

## Preliminary EDA
The prelimary eda for check-in 2 will consists of preparation of the training dataset.  This involves organization of the proteins into a unique set of pairs of proteins that interact and pairs of proteins that do not interact.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests

The commented code below includes saved functions that query the RCSB Protein DataBase. I originally scraped from this site, but ended up switching to UniProt. I've kept the code below, just in case I want to gather corroborating data later on.

In [2]:
# # This function is adapted from Carlos Oliver
# # https://www.rcsb.org/pages/webservices/python3_search_nmr

# def experimental_query(method):
#     url = 'http://www.rcsb.org/pdb/rest/search'

#     query_text = """
# <?xml version="1.0" encoding="UTF-8"?>

# <orgPdbQuery>

# <version>B0907</version>

# <queryType>org.pdb.query.simple.ExpTypeQuery</queryType>

# <description>Experimental Method Search: Experimental Method="""+method+"""</description>

# <mvStructure.expMethod.value>"""+method+"""</mvStructure.expMethod.value>

# </orgPdbQuery>

# """
#     query_helper(query_text)

# def query_helper(query_text):    

#     print("Query: %s" % query_text)
#     print("Querying RCSB PDB REST API...")

#     header = {'Content-Type': 'application/x-www-form-urlencoded'}

#     response = requests.post(url, data=query_text, headers=header)

#     if response.status_code == 200:
#         print("Found %d PDB entries matching query." % len(response.text))
#         print("Matches: \n%s" % response.text)
#     else:
#         print("Failed to retrieve results")

In [3]:
# solid_state = "SOLID-STATE NMR"
# experimental_query(solid_state)

In [4]:
train = pd.read_excel("./train_set.xlsx")
test = pd.read_excel("./test_set.xlsx")

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11935 entries, 0 to 11934
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Entry           11935 non-null  object
 1   Entry name      11935 non-null  object
 2   Protein names   11935 non-null  object
 3   Interacts with  11935 non-null  object
 4   Sequence        11935 non-null  object
 5   Mass            11935 non-null  object
 6   Length          11935 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 652.8+ KB


In [6]:
train.head()

,Entry,Entry name,Protein names,Interacts with,Sequence,Mass,Length
0,Q13023,AKAP6_HUMAN,A-kinase anchor protein 6 (AKAP-6) (A-kinase a...,Q96CV9,MLTMSVTLSPLRSQDLDPMATDASPMAINMTPTVEQGEGEEAMKDM...,"256,720",2319
1,Q8WTP8,AEN_HUMAN,Apoptosis-enhancing nuclease (EC 3.1.-.-) (Int...,C9JG97; P50402; Q96B26; Q3T906; Q8IX15-3; Q134...,MVPREAPESAQCLCPSLTIPNAKDVLRKRHKRRSRQHQRFMARKAL...,"36,350",325
2,Q9NP61,ARFG3_HUMAN,ADP-ribosylation factor GTPase-activating prot...,Q9H400; Q8WWV3; Q16563; Q8IUH5,MGDPSKQDILTIFKRLRSVPTNKVCFDCGAKNPSWASITYGVFLCI...,"56,928",516
3,P36404,ARL2_HUMAN,ADP-ribosylation factor-like protein 2,Q9Y2Y0; Q969G2; O43924; Q9BZK7; Q13432,MGLLTILKKMKQKERELRLLMLGLDNAGKTTILKKFNGEDIDTISP...,"20,878",184
4,Q9NQ90,ANO2_HUMAN,Anoctamin-2 (Transmembrane protein 16B),Q12959,MATPGPRDIPLLPGSPRRLSPQAGSRGGQGPKHGQQCLKMPGPRAP...,"113,969",1003


In [7]:
# Let's look at the training dataset
train.shape

(11935, 7)

There are 11935 proteins in our training dataset.

In [8]:
train.set_index("Entry", inplace=True)
train.head()

,Entry name,Protein names,Interacts with,Sequence,Mass,Length
Entry,,,,,,
Q13023,AKAP6_HUMAN,A-kinase anchor protein 6 (AKAP-6) (A-kinase a...,Q96CV9,MLTMSVTLSPLRSQDLDPMATDASPMAINMTPTVEQGEGEEAMKDM...,"256,720",2319
Q8WTP8,AEN_HUMAN,Apoptosis-enhancing nuclease (EC 3.1.-.-) (Int...,C9JG97; P50402; Q96B26; Q3T906; Q8IX15-3; Q134...,MVPREAPESAQCLCPSLTIPNAKDVLRKRHKRRSRQHQRFMARKAL...,"36,350",325
Q9NP61,ARFG3_HUMAN,ADP-ribosylation factor GTPase-activating prot...,Q9H400; Q8WWV3; Q16563; Q8IUH5,MGDPSKQDILTIFKRLRSVPTNKVCFDCGAKNPSWASITYGVFLCI...,"56,928",516
P36404,ARL2_HUMAN,ADP-ribosylation factor-like protein 2,Q9Y2Y0; Q969G2; O43924; Q9BZK7; Q13432,MGLLTILKKMKQKERELRLLMLGLDNAGKTTILKKFNGEDIDTISP...,"20,878",184
Q9NQ90,ANO2_HUMAN,Anoctamin-2 (Transmembrane protein 16B),Q12959,MATPGPRDIPLLPGSPRRLSPQAGSRGGQGPKHGQQCLKMPGPRAP...,"113,969",1003


In [9]:
train["Interacts with"] = train["Interacts with"].apply(lambda a: [b.strip() for b in a.split(";")])

In [10]:
train.head()

,Entry name,Protein names,Interacts with,Sequence,Mass,Length
Entry,,,,,,
Q13023,AKAP6_HUMAN,A-kinase anchor protein 6 (AKAP-6) (A-kinase a...,[Q96CV9],MLTMSVTLSPLRSQDLDPMATDASPMAINMTPTVEQGEGEEAMKDM...,"256,720",2319
Q8WTP8,AEN_HUMAN,Apoptosis-enhancing nuclease (EC 3.1.-.-) (Int...,"[C9JG97, P50402, Q96B26, Q3T906, Q8IX15-3, Q13...",MVPREAPESAQCLCPSLTIPNAKDVLRKRHKRRSRQHQRFMARKAL...,"36,350",325
Q9NP61,ARFG3_HUMAN,ADP-ribosylation factor GTPase-activating prot...,"[Q9H400, Q8WWV3, Q16563, Q8IUH5]",MGDPSKQDILTIFKRLRSVPTNKVCFDCGAKNPSWASITYGVFLCI...,"56,928",516
P36404,ARL2_HUMAN,ADP-ribosylation factor-like protein 2,"[Q9Y2Y0, Q969G2, O43924, Q9BZK7, Q13432]",MGLLTILKKMKQKERELRLLMLGLDNAGKTTILKKFNGEDIDTISP...,"20,878",184
Q9NQ90,ANO2_HUMAN,Anoctamin-2 (Transmembrane protein 16B),[Q12959],MATPGPRDIPLLPGSPRRLSPQAGSRGGQGPKHGQQCLKMPGPRAP...,"113,969",1003


In [11]:
interacting_pairs = set()
all_proteins = set()

for protein1 in train.index:
    all_proteins.add(protein1)
    
    for protein2 in train.loc[protein1,'Interacts with']:
        all_proteins.add(protein2)
        interacting_pairs.add(frozenset([protein1,protein2]))

In [12]:
interact_df = pd.DataFrame({"proteins" : list(interacting_pairs), "interacts" : 1})

In [13]:
interact_df.head()

,proteins,interacts
0,"(Q13490, P14136)",1
1,"(Q14525, P17661)",1
2,"(Q96DT7, Q70IA8)",1
3,"(Q9BSJ6, Q96DZ9-2)",1
4,"(Q9UER7, P14921-1)",1


In [14]:
def generate_noninteractors(n):
    noninteractors = set()
    while len(noninteractors) < n:
        pair_pick = frozenset(np.random.choice(list(all_proteins), 2, replace=False))
        if (pair_pick not in interacting_pairs):
            noninteractors.add(pair_pick)
    return noninteractors

In [15]:
noninteracting_pairs = list(generate_noninteractors(interact_df.shape[0]))

In [16]:
noninteract_df = pd.DataFrame({"proteins" : noninteracting_pairs, "interacts" : 0})

In [17]:
noninteract_df

,proteins,interacts
0,"(P01135, Q96QC0)",0
1,"(Q9Y221, O00422)",0
2,"(Q6NUT3, Q04118)",0
3,"(Q86SQ0, Q96J77)",0
4,"(D6RGH6, A1L4H1)",0
...,...,...
89410,"(Q6PK04, Q62101)",0
89411,"(Q8BGY9, Q9BTD8)",0
89412,"(P60228, Q13362-3)",0
89413,"(Q9Y3I1-1, P12268)",0


In [18]:
train_clean = pd.concat([interact_df,noninteract_df])
train_clean.set_index("proteins", inplace=True)

In [19]:
train_clean

,interacts
proteins,
"(Q13490, P14136)",1
"(Q14525, P17661)",1
"(Q96DT7, Q70IA8)",1
"(Q9BSJ6, Q96DZ9-2)",1
"(Q9UER7, P14921-1)",1
...,...
"(Q6PK04, Q62101)",0
"(Q8BGY9, Q9BTD8)",0
"(P60228, Q13362-3)",0


In [20]:
i = 0
train_clean["seq1"] = np.nan
train_clean["seq2"] = np.nan

for pair in train_clean.index:
    try:
        train_clean.iloc[i,1] = train.loc[list(pair)[0],"Sequence"]
        train_clean.iloc[i,2] = train.loc[list(pair)[1],"Sequence"]
    except:
        pass
    i += 1

In [21]:
train_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 178830 entries, frozenset({'Q13490', 'P14136'}) to frozenset({'Q8TF76', 'A0A0S2Z5F2'})
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   interacts  178830 non-null  int64 
 1   seq1       143320 non-null  object
 2   seq2       114803 non-null  object
dtypes: int64(1), object(2)
memory usage: 5.5+ MB


In [22]:
train_clean.dropna(inplace=True)
train_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 114803 entries, frozenset({'Q13490', 'P14136'}) to frozenset({'Q14585', 'Q96KM6'})
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   interacts  114803 non-null  int64 
 1   seq1       114803 non-null  object
 2   seq2       114803 non-null  object
dtypes: int64(1), object(2)
memory usage: 3.5+ MB


After cleaning and preparation.  The training dataset has ~115,000 pairs of proteins.

In [23]:
train_clean.to_csv("./data/train_clean.csv")